In [ ]:
#| hide
#| eval: false
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [ ]:
#| export
from __future__ import annotations
from fastai.basics import *

In [ ]:
#| hide
from nbdev.showdoc import *
import math

In [ ]:
#| default_exp callback.hook

# Model hooks

> Callback and helper function to add hooks in models

In [ ]:
from fastai.test_utils import *

## What are hooks?

Hooks are functions you can attach to a particular layer in your model and that will be executed in the forward pass (for forward hooks) or backward pass (for backward hooks). Here we begin with an introduction around hooks, but you should jump to `HookCallback` if you quickly want to implement one (and read the following example `ActivationStats`).

Forward hooks are functions that take three arguments: the layer it's applied to, the input of that layer and the output of that layer.

In [ ]:
tst_model = nn.Linear(5,3)
def example_forward_hook(m,i,o): print(m,i,o)
    
x = torch.randn(4,5)
hook = tst_model.register_forward_hook(example_forward_hook)
y = tst_model(x)
hook.remove()

Linear(in_features=5, out_features=3, bias=True) (tensor([[-0.9811,  0.1455,  0.3667,  0.7821,  1.0376],
        [ 0.4916, -0.8581,  0.1134,  0.1752, -0.0595],
        [ 0.4517, -0.9027,  1.3693, -0.8399,  1.4931],
        [-0.7818, -1.1915, -0.1014,  1.1878, -0.8517]]),) tensor([[-0.1019, -0.4006, -0.3282],
        [-0.0551,  0.5754,  0.0726],
        [-0.5382, -0.1731, -0.1683],
        [-0.3195,  0.7669,  0.3924]], grad_fn=<AddmmBackward0>)


Backward hooks are functions that take three arguments: the layer it's applied to, the gradients of the loss with respect to the input, and the gradients with respect to the output.

In [ ]:
def example_backward_hook(m,gi,go): print(m,gi,go)
hook = tst_model.register_backward_hook(example_backward_hook)

x = torch.randn(4,5)
y = tst_model(x)
loss = y.pow(2).mean()
loss.backward()
hook.remove()

Linear(in_features=5, out_features=3, bias=True) (tensor([ 0.0913,  0.3834, -0.0015]), None, tensor([[ 0.1872,  0.1248, -0.2946],
        [ 0.1090, -0.3164, -0.2486],
        [-0.0468, -0.1728, -0.1686],
        [-0.0787,  0.3200,  0.0099],
        [-0.0308, -0.1119,  0.0056]])) (tensor([[ 0.0414,  0.1750,  0.0672],
        [-0.0252,  0.0636,  0.0592],
        [ 0.1243,  0.0364, -0.1118],
        [-0.0491,  0.1084, -0.0160]]),)


/home/benja/.conda/envs/fastaidev/lib/python3.12/site-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Hooks can change the input/output of a layer, or the gradients, print values or shapes. If you want to store something related to theses inputs/outputs, it's best to have your hook associated to a class so that it can put it in the state of an instance of that class.

## Hook -

In [ ]:
#| export
@docs
class Hook():
    "Create a hook on `m` with `hook_func`."
    def __init__(self, m, hook_func, is_forward=True, detach=True, cpu=False, gather=False):
        store_attr('hook_func,detach,cpu,gather')
        f = m.register_forward_hook if is_forward else m.register_backward_hook
        self.hook = f(self.hook_fn)
        self.stored,self.removed = None,False

    def hook_fn(self, module, input, output):
        "Applies `hook_func` to `module`, `input`, `output`."
        if self.detach:
            input,output = to_detach(input, cpu=self.cpu, gather=self.gather),to_detach(output, cpu=self.cpu, gather=self.gather)
        self.stored = self.hook_func(module, input, output)

    def remove(self):
        "Remove the hook from the model."
        if not self.removed:
            self.hook.remove()
            self.removed=True

    def __enter__(self, *args): return self
    def __exit__(self, *args): self.remove()

    _docs = dict(__enter__="Register the hook",
                 __exit__="Remove the hook")

This will be called during the forward pass if `is_forward=True`, the backward pass otherwise, and will optionally `detach`, `gather` and put on the `cpu` the (gradient of the) input/output of the model before passing them to `hook_func`. The result of `hook_func` will be stored in the `stored` attribute of the `Hook`.

In [ ]:
tst_model = nn.Linear(5,3)
hook = Hook(tst_model, lambda m,i,o: o)
y = tst_model(x)
test_eq(hook.stored, y)

In [ ]:
show_doc(Hook.hook_fn)

---

[source](https://github.com/fastai/fastai/blob/main/fastai/callback/hook.py#L23){target="_blank" style="float:right; font-size:smaller"}

### Hook.hook_fn

>      Hook.hook_fn (module, input, output)

*Applies `hook_func` to `module`, `input`, `output`.*

In [ ]:
show_doc(Hook.remove)

---

[source](https://github.com/fastai/fastai/blob/main/fastai/callback/hook.py#L29){target="_blank" style="float:right; font-size:smaller"}

### Hook.remove

>      Hook.remove ()

*Remove the hook from the model.*

:::{.callout-note}

It's important to properly remove your hooks for your model when you're done to avoid them being called again next time your model is applied to some inputs, and to free the memory that go with their state.

:::

In [ ]:
tst_model = nn.Linear(5,10)
x = torch.randn(4,5)
y = tst_model(x)
hook = Hook(tst_model, example_forward_hook)
test_stdout(lambda: tst_model(x), f"{tst_model} ({x},) {y.detach()}")
hook.remove()
test_stdout(lambda: tst_model(x), "")

### Context Manager

Since it's very important to remove your `Hook` even if your code is interrupted by some bug, `Hook` can be used as context managers.

In [ ]:
show_doc(Hook.__enter__)

---

[source](https://github.com/fastai/fastai/blob/main/fastai/callback/hook.py#L35){target="_blank" style="float:right; font-size:smaller"}

### Hook.__enter__

>      Hook.__enter__ (*args)

*Register the hook*

In [ ]:
show_doc(Hook.__exit__)

---

[source](https://github.com/fastai/fastai/blob/main/fastai/callback/hook.py#L36){target="_blank" style="float:right; font-size:smaller"}

### Hook.__exit__

>      Hook.__exit__ (*args)

*Remove the hook*

In [ ]:
tst_model = nn.Linear(5,10)
x = torch.randn(4,5)
y = tst_model(x)
with Hook(tst_model, example_forward_hook) as h:
    test_stdout(lambda: tst_model(x), f"{tst_model} ({x},) {y.detach()}")
test_stdout(lambda: tst_model(x), "")

In [ ]:
#| export
def _hook_inner(m,i,o): return o if isinstance(o,Tensor) or is_listy(o) else list(o)

def hook_output(module, detach=True, cpu=False, grad=False):
    "Return a `Hook` that stores activations of `module` in `self.stored`"
    return Hook(module, _hook_inner, detach=detach, cpu=cpu, is_forward=not grad)

The activations stored are the gradients if `grad=True`, otherwise the output of `module`. If `detach=True` they are detached from their history, and if `cpu=True`, they're put on the CPU.

In [ ]:
tst_model = nn.Linear(5,10)
x = torch.randn(4,5)
with hook_output(tst_model) as h:
    y = tst_model(x)
    test_eq(y, h.stored)
    assert not h.stored.requires_grad
    
with hook_output(tst_model, grad=True) as h:
    y = tst_model(x)
    loss = y.pow(2).mean()
    loss.backward()
    test_close(2*y / y.numel(), h.stored[0])

In [ ]:
#| cuda
with hook_output(tst_model, cpu=True) as h:
    y = tst_model.cuda()(x.cuda())
    test_eq(h.stored.device, torch.device('cpu'))

## Hooks -

In [ ]:
#| export
@docs
class Hooks():
    "Create several hooks on the modules in `ms` with `hook_func`."
    def __init__(self, ms, hook_func, is_forward=True, detach=True, cpu=False):
        self.hooks = [Hook(m, hook_func, is_forward, detach, cpu) for m in ms]

    def __getitem__(self,i): return self.hooks[i]
    def __len__(self):       return len(self.hooks)
    def __iter__(self):      return iter(self.hooks)
    @property
    def stored(self):        return L(o.stored for o in self)

    def remove(self):
        "Remove the hooks from the model."
        for h in self.hooks: h.remove()

    def __enter__(self, *args): return self
    def __exit__ (self, *args): self.remove()

    _docs = dict(stored = "The states saved in each hook.",
                 __enter__="Register the hooks",
                 __exit__="Remove the hooks")

In [ ]:
layers = [nn.Linear(5,10), nn.ReLU(), nn.Linear(10,3)]
tst_model = nn.Sequential(*layers)
hooks = Hooks(tst_model, lambda m,i,o: o)
y = tst_model(x)
test_eq(hooks.stored[0], layers[0](x))
test_eq(hooks.stored[1], F.relu(layers[0](x)))
test_eq(hooks.stored[2], y)
hooks.remove()

In [ ]:
show_doc(Hooks.stored, name='Hooks.stored')

---

[source](https://github.com/fastai/fastai/blob/main/fastai/callback/hook.py#L59){target="_blank" style="float:right; font-size:smaller"}

### Hooks.stored

>      Hooks.stored ()

In [ ]:
show_doc(Hooks.remove)

---

[source](https://github.com/fastai/fastai/blob/main/fastai/callback/hook.py#L61){target="_blank" style="float:right; font-size:smaller"}

### Hooks.remove

>      Hooks.remove ()

*Remove the hooks from the model.*

### Context Manager

Like `Hook` , you can use `Hooks` as context managers.

In [ ]:
show_doc(Hooks.__enter__)

---

[source](https://github.com/fastai/fastai/blob/main/fastai/callback/hook.py#L65){target="_blank" style="float:right; font-size:smaller"}

### Hooks.__enter__

>      Hooks.__enter__ (*args)

*Register the hooks*

In [ ]:
show_doc(Hooks.__exit__)

---

[source](https://github.com/fastai/fastai/blob/main/fastai/callback/hook.py#L66){target="_blank" style="float:right; font-size:smaller"}

### Hooks.__exit__

>      Hooks.__exit__ (*args)

*Remove the hooks*

In [ ]:
layers = [nn.Linear(5,10), nn.ReLU(), nn.Linear(10,3)]
tst_model = nn.Sequential(*layers)
with Hooks(layers, lambda m,i,o: o) as h:
    y = tst_model(x)
    test_eq(h.stored[0], layers[0](x))
    test_eq(h.stored[1], F.relu(layers[0](x)))
    test_eq(h.stored[2], y)

In [ ]:
#| export
def hook_outputs(modules, detach=True, cpu=False, grad=False):
    "Return `Hooks` that store activations of all `modules` in `self.stored`"
    return Hooks(modules, _hook_inner, detach=detach, cpu=cpu, is_forward=not grad)

The activations stored are the gradients if `grad=True`, otherwise the output of `modules`. If `detach=True` they are detached from their history, and if `cpu=True`, they're put on the CPU.

In [ ]:
layers = [nn.Linear(5,10), nn.ReLU(), nn.Linear(10,3)]
tst_model = nn.Sequential(*layers)
x = torch.randn(4,5)
with hook_outputs(layers) as h:
    y = tst_model(x)
    test_eq(h.stored[0], layers[0](x))
    test_eq(h.stored[1], F.relu(layers[0](x)))
    test_eq(h.stored[2], y)
    for s in h.stored: assert not s.requires_grad
    
with hook_outputs(layers, grad=True) as h:
    y = tst_model(x)
    loss = y.pow(2).mean()
    loss.backward()
    g = 2*y / y.numel()
    test_close(g, h.stored[2][0])
    g = g @ layers[2].weight.data
    test_close(g, h.stored[1][0])
    g = g * (layers[0](x) > 0).float()
    test_close(g, h.stored[0][0])

In [ ]:
#| cuda
with hook_outputs(tst_model, cpu=True) as h:
    y = tst_model.cuda()(x.cuda())
    for s in h.stored: test_eq(s.device, torch.device('cpu'))

In [ ]:
#| export
def dummy_eval(m, size=(64,64)):
    "Evaluate `m` on a dummy input of a certain `size`"
    ch_in = in_channels(m)
    x = one_param(m).new(1, ch_in, *size).requires_grad_(False).uniform_(-1.,1.)
    with torch.no_grad(): return m.eval()(x)

In [ ]:
#| export
def model_sizes(m, size=(64,64)):
    "Pass a dummy input through the model `m` to get the various sizes of activations."
    with hook_outputs(m) as hooks:
        _ = dummy_eval(m, size=size)
        return [o.stored.shape for o in hooks]

In [ ]:
m = nn.Sequential(ConvLayer(3, 16), ConvLayer(16, 32, stride=2), ConvLayer(32, 32))
test_eq(model_sizes(m), [[1, 16, 64, 64], [1, 32, 32, 32], [1, 32, 32, 32]])

In [ ]:
#| export
def num_features_model(m):
    "Return the number of output features for `m`."
    sz,ch_in = 32,in_channels(m)
    while True:
        #Trying for a few sizes in case the model requires a big input size.
        try:
            return model_sizes(m, (sz,sz))[-1][1]
        except Exception as e:
            sz *= 2
            if sz > 2048: raise e

In [ ]:
m = nn.Sequential(nn.Conv2d(5,4,3), nn.Conv2d(4,3,3))
test_eq(num_features_model(m), 3)
m = nn.Sequential(ConvLayer(3, 16), ConvLayer(16, 32, stride=2), ConvLayer(32, 32))
test_eq(num_features_model(m), 32)

## HookCallback -

To make hooks easy to use, we wrapped a version in a Callback where you just have to implement a `hook` function (plus any element you might need).

In [ ]:
#| export
def has_params(m):
    "Check if `m` has at least one parameter"
    return len(list(m.parameters())) > 0

In [ ]:
assert has_params(nn.Linear(3,4))
assert has_params(nn.LSTM(4,5,2))
assert not has_params(nn.ReLU())

In [ ]:
#| export
@funcs_kwargs
class HookCallback(Callback):
    "`Callback` that can be used to register hooks on `modules`"
    _methods = ["hook"]
    hook = noops
    def __init__(self, modules=None, every=None, remove_end=True, is_forward=True, detach=True, cpu=True, include_paramless=False , **kwargs):
        store_attr('modules,every,remove_end,is_forward,detach,cpu, include_paramless')
        assert not kwargs

    def before_fit(self):
        "Register the `Hooks` on `self.modules`."
        if self.modules is None: self.modules = [m for m in flatten_model(self.model) if self.include_paramless or has_params(m)]
        if self.every is None: self._register()

    def before_batch(self):
        if self.every is None: return
        if self.training and self.train_iter%self.every==0: self._register()

    def after_batch(self):
        if self.every is None: return
        if self.training and self.train_iter%self.every==0: self._remove()

    def after_fit(self):
        "Remove the `Hooks`."
        if self.remove_end: self._remove()

    def _register(self): self.hooks = Hooks(self.modules, self.hook, self.is_forward, self.detach, self.cpu)
    def _remove(self):
        if getattr(self, 'hooks', None): self.hooks.remove()

    def __del__(self): self._remove()

You can either subclass and implement a `hook` function (along with any event you want) or pass that a `hook` function when initializing. Such a function needs to take three argument: a layer, input and output (for a backward hook, input means gradient with respect to the inputs, output, gradient with respect to the output) and can either modify them or update the state according to them.

If not provided, `modules` will default to the layers of `self.model` that have a `weight` attribute. (to include layers of `self.model` that *do not* have a `weight` attribute e.g `ReLU`, `Flatten` etc., set `include_paramless=True`)
Depending on `do_remove`, the hooks will be properly removed at the end of training (or in case of error). `is_forward` , `detach` and `cpu` are passed to `Hooks`.

The function called at each forward (or backward) pass is `self.hook` and must be implemented when subclassing this callback.

In [ ]:
class TstCallback(HookCallback):
    def hook(self, m, i, o): return o
    def after_batch(self): test_eq(self.hooks.stored[0], self.pred)
        
learn = synth_learner(n_trn=5, cbs = TstCallback())
learn.fit(1)

[0, 6.587433815002441, 5.402360916137695, '00:00']


/home/benja/fastai/fastai/fastai/callback/core.py:71: UserWarning: You are shadowing an attribute (modules) that exists in the learner. Use `self.learn.modules` to avoid this
  warn(f"You are shadowing an attribute ({name}) that exists in the learner. Use `self.learn.{name}` to avoid this")


In [ ]:
class TstCallback(HookCallback):
    def __init__(self, modules=None, remove_end=True, detach=True, cpu=False):
        super().__init__(modules, None, remove_end, False, detach, cpu)
    def hook(self, m, i, o): return o
    def after_batch(self):
        if self.training:
            test_eq(self.hooks.stored[0][0], 2*(self.pred-self.y)/self.pred.shape[0])
        
learn = synth_learner(n_trn=5, cbs = TstCallback())
learn.fit(1)

[0, 8.743090629577637, 10.072294235229492, '00:00']


In [ ]:
show_doc(HookCallback.before_fit)

---

[source](https://github.com/fastai/fastai/blob/main/fastai/callback/hook.py#L118){target="_blank" style="float:right; font-size:smaller"}

### HookCallback.before_fit

>      HookCallback.before_fit ()

*Register the `Hooks` on `self.modules`.*

In [ ]:
show_doc(HookCallback.after_fit)

---

[source](https://github.com/fastai/fastai/blob/main/fastai/callback/hook.py#L131){target="_blank" style="float:right; font-size:smaller"}

### HookCallback.after_fit

>      HookCallback.after_fit ()

*Remove the `Hooks`.*

## Model summary

In [ ]:
#| export
def total_params(m):
    "Give the number of parameters of a module and if it's trainable or not"
    params = sum([p.numel() for p in m.parameters()])
    trains = [p.requires_grad for p in m.parameters()]
    return params, (False if len(trains)==0 else trains[0])

In [ ]:
test_eq(total_params(nn.Linear(10,32)), (32*10+32,True))
test_eq(total_params(nn.Linear(10,32, bias=False)), (32*10,True))
test_eq(total_params(nn.BatchNorm2d(20)), (20*2, True))
test_eq(total_params(nn.BatchNorm2d(20, affine=False)), (0,False))
test_eq(total_params(nn.Conv2d(16, 32, 3)), (16*32*3*3 + 32, True))
test_eq(total_params(nn.Conv2d(16, 32, 3, bias=False)), (16*32*3*3, True))
#First ih layer 20--10, all else 10--10. *4 for the four gates
test_eq(total_params(nn.LSTM(20, 10, 2)), (4 * (20*10 + 10) + 3 * 4 * (10*10 + 10), True))

In [ ]:
#| export
def layer_info(learn, *xb):
    "Return layer infos of `model` on `xb` (only support batch first inputs)"
    def _track(m, i, o): 
        params, trainable, shape = '', '', ''
        same = any((isinstance(x[0], torch.Tensor) and x[0].shape[1:] == x[1].shape for x in zip(i, o)))
        shape = apply(lambda x: x.shape, o)
        if hasattr(m, 'weight'): # non activation layer
            params, trainable = total_params(m)
        return (type(m).__name__, params, trainable, shape, same)
            
    with Hooks(flatten_model(learn.model), _track) as h:
        batch = apply(lambda o:o[:1], xb)
        train_only_cbs = [cb for cb in learn.cbs if hasattr(cb, '_only_train_loop')]
        with learn.removed_cbs(train_only_cbs), learn.no_logging(), learn as l:
            r = l.get_preds(dl=[batch], inner=True, reorder=False)
        return h.stored

The output of `_track` is expected to be a `tuple` of module name, the number of parameters, the shape of the layer, whether it is trainable, what layer group it belongs to, and whether or not the size changed. There are three potential groups that can show:

* A non-activation layer (Linear, Conv, etc)
* An activation layer
* A pooling layer

Depending on which only part of the output is really returned, otherwise it is `''`. For non-activation layers everything is returned. Activation layers only return a name, the shape and `False` for `same`. Pooling layers will return the name, the new shape, and `False` for `same`

In [ ]:
def _m(): return nn.Sequential(nn.Linear(1,50), nn.ReLU(), nn.BatchNorm1d(50), nn.Linear(50, 1))
sample_input = torch.randn((16, 1))
test_eq(layer_info(synth_learner(model=_m()), sample_input), [
    ('Linear', 100, True, [1, 50], False),
    ('ReLU', '', '', [1,50], True),
    ('BatchNorm1d', 100, True, [1, 50], True),
    ('Linear', 51, True, [1, 1], False)
])

In [ ]:
#| hide
# Test for Flatten
def _tst_m(): return nn.Sequential(
    nn.Conv2d(1, 2, kernel_size=3, padding=1, stride=2),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(8,50), 
    nn.ReLU(), 
    nn.BatchNorm1d(50), 
    nn.Linear(50, 1)
)
                                                              
sample_input = torch.randn((1,1,4,4))
test_eq(layer_info(synth_learner(model=_tst_m()), sample_input), [
    ('Conv2d', 20, True, [1, 2, 2, 2], False),
    ('ReLU', '', '', [1, 2, 2, 2], True),
    ('Flatten', '', '', [1, 8], False),
    ('Linear', 450, True, [1, 50], False),
    ('ReLU', '', '', [1,50], True),
    ('BatchNorm1d', 100, True, [1, 50], True),
    ('Linear', 51, True, [1, 1], False)
])

In [ ]:
#| hide
# Test for multiple inputs model
class _2InpModel(Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(nn.Linear(2,50), nn.ReLU(), nn.BatchNorm1d(50), nn.Linear(50, 1))
    def forward(self, *inps):
        outputs = torch.cat(inps, dim=-1)
        return self.seq(outputs)

sample_inputs = (torch.randn(16, 1), torch.randn(16, 1))
learn = synth_learner(model=_2InpModel())
learn.dls.n_inp = 2
test_eq(layer_info(learn, *sample_inputs), [
    ('Linear', 150, True, [1, 50], False),
    ('ReLU', '', '', [1,50], True),
    ('BatchNorm1d', 100, True, [1, 50], True),
    ('Linear', 51, True, [1, 1], False)
])

In [ ]:
#| export
def _get_shapes(o, bs): 
    inp = o[first(o)] if (isinstance(o, dict)) else o
    return ' x '.join([str(bs)] + [str(t) for t in inp[1:]])

def _print_shapes(o, bs):
    if isinstance(o, torch.Size): return _get_shapes(o, bs)
    elif isinstance(o, tuple): return _get_shapes(o[0], bs)
    else: return str([_print_shapes(x, bs) for x in o])

In [ ]:
#| export
def module_summary(learn, *xb):
    "Print a summary of `model` using `xb`"
    #Individual parameters wrapped in ParameterModule aren't called through the hooks in `layer_info`,
    #  thus are not counted inside the summary
    #TODO: find a way to have them counted in param number somehow
    infos = layer_info(learn, *xb)
    n,bs = 76,find_bs(xb)
    inp_sz = _print_shapes(apply(lambda x:x.shape, xb), bs)
    res = f"{type(learn.model).__name__} (Input shape: {inp_sz})\n"
    res += "=" * n + "\n"
    res += f"{'Layer (type)':<20} {'Output Shape':<20} {'Param #':<10} {'Trainable':<10}\n"
    res += "=" * n
    ps,trn_ps,j = 0,0,0
    infos = [o for o in infos if o is not None] #see comment in previous cell
    prev_sz = None
    for typ,np,trn,sz,chnged in infos:
        if sz is None: continue
        if j == 0:
            res += f'\n{"":<20} {_print_shapes(sz, bs)[:19]:<20}' # to avoid a double line at the top
        if not chnged and not prev_sz == sz and j > 0: res += "\n" + "_" * n + "\n" + f'{"":<20} {_print_shapes(sz, bs)[:19]:<20}'
        j = 1
        res += f"\n{typ:<20} {'':<20} {np:<10} {str(trn):<10}"
        if np != '':
            ps += np
            if trn: trn_ps += np
        prev_sz = sz
    res += "\n" + "_" * n + "\n"
    res += f"\nTotal params: {ps:,}\n"
    res += f"Total trainable params: {trn_ps:,}\n"
    res += f"Total non-trainable params: {ps - trn_ps:,}\n\n"
    return PrettyString(res)

In [ ]:
#| export
@patch
def summary(self:Learner):
    "Print a summary of the model, optimizer and loss function."
    xb = self.dls.train.one_batch()[:getattr(self.dls.train, "n_inp", 1)]
    res = module_summary(self, *xb)
    res += f"Optimizer used: {self.opt_func}\nLoss function: {self.loss_func}\n\n"
    if self.opt is not None:
        res += f"Model " + ("unfrozen\n\n" if self.opt.frozen_idx==0 else f"frozen up to parameter group #{self.opt.frozen_idx}\n\n")
    res += "Callbacks:\n" + '\n'.join(f"  - {cb}" for cb in self.cbs.sorted('order'))
    return PrettyString(res)

In [ ]:
learn = synth_learner(model=_m())
learn.summary()

Sequential (Input shape: 16 x 1)
Layer (type)         Output Shape         Param #    Trainable 
                     16 x 50             
Linear                                    100        True      
ReLU                                                           
BatchNorm1d                               100        True      
____________________________________________________________________________
                     16 x 1              
Linear                                    51         True      
____________________________________________________________________________

Total params: 251
Total trainable params: 251
Total non-trainable params: 0

Optimizer used: functools.partial(<function SGD>, mom=0.9)
Loss function: FlattenedLoss of MSELoss()

Callbacks:
  - TrainEvalCallback
  - CastToTensor
  - Recorder

In [ ]:
#| hide
#| cuda
learn = synth_learner(model=_m(), cuda=True)
learn.summary()

Sequential (Input shape: 16 x 1)
Layer (type)         Output Shape         Param #    Trainable 
                     16 x 50             
Linear                                    100        True      
ReLU                                                           
BatchNorm1d                               100        True      
____________________________________________________________________________
                     16 x 1              
Linear                                    51         True      
____________________________________________________________________________

Total params: 251
Total trainable params: 251
Total non-trainable params: 0

Optimizer used: functools.partial(<function SGD>, mom=0.9)
Loss function: FlattenedLoss of MSELoss()

Callbacks:
  - TrainEvalCallback
  - CastToTensor
  - Recorder

In [ ]:
#| hide
# Test for multiple output
class _NOutModel(Module):
    def __init__(self): self.lin = nn.Linear(5, 6)
    def forward(self, x1):
        x = torch.randn((10, 5))
        return x,self.lin(x)

learn = synth_learner(model = _NOutModel())
learn.summary() # Output Shape should be (50, 16, 256), (1, 16, 256)

_NOutModel (Input shape: 16 x 1)
Layer (type)         Output Shape         Param #    Trainable 
                     16 x 6              
Linear                                    36         True      
____________________________________________________________________________

Total params: 36
Total trainable params: 36
Total non-trainable params: 0

Optimizer used: functools.partial(<function SGD>, mom=0.9)
Loss function: FlattenedLoss of MSELoss()

Callbacks:
  - TrainEvalCallback
  - CastToTensor
  - Recorder

In [ ]:
#| hide
# Test for the case (as in Book) when learn.dls.train_ds is a list not fastai.data.core.Datasets
train_x = torch.rand((100, 4))
train_y = torch.rand((100, 1))

valid_x = torch.rand((100, 4))
valid_y = torch.rand((100,1))

dset = list(zip(train_x,train_y))
valid_dset = list(zip(valid_x,valid_y))

dl = DataLoader(dset, batch_size=16)
val_dl = DataLoader(valid_dset, batch_size=16)

dls = DataLoaders(dl, val_dl)

simple_net = nn.Sequential(
    nn.Linear(4, 2),
    nn.ReLU(),
    nn.Linear(2,1)
)
learn = Learner(dls, simple_net, loss_func=F.l1_loss)
learn.summary()

Sequential (Input shape: 16 x 4)
Layer (type)         Output Shape         Param #    Trainable 
                     16 x 2              
Linear                                    10         True      
ReLU                                                           
____________________________________________________________________________
                     16 x 1              
Linear                                    3          True      
____________________________________________________________________________

Total params: 13
Total trainable params: 13
Total non-trainable params: 0

Optimizer used: <function Adam>
Loss function: <function l1_loss>

Callbacks:
  - TrainEvalCallback
  - CastToTensor
  - Recorder

## Activation graphs

In [ ]:
#| export
@delegates()
class ActivationStats(HookCallback):
    "Callback that record the mean and std of activations."
    order=-20
    def __init__(self, with_hist=False, **kwargs):
        super().__init__(**kwargs)
        self.with_hist = with_hist

    def before_fit(self):
        "Initialize stats."
        super().before_fit()
        self.stats = L()

    def hook(self, m, i, o):
        if isinstance(o, tuple): return self.hook_multi_ouput(o)
        o = o.float()
        res = {'mean': o.mean().item(), 'std': o.std().item(),
               'near_zero': (o<=0.05).long().sum().item()/o.numel()}
        if self.with_hist: res['hist'] = o.histc(40,0,10)
        return res
    
    def hook_multi_ouput(self,o_tuple):
        "For outputs of RNN which are [nested] tuples of tensors"
        res = []
        for o in self._flatten_tuple(o_tuple):
            if not(isinstance(o, Tensor)): continue
            res.append(self.hook(None, None, o))
        return res

    def _flatten_tuple(self, o_tuple):
        "Recursively flatten a [nested] tuple"
        res = []
        for it in o_tuple:
            if isinstance(it, tuple): res += self._flatten_tuple(it)
            else: res += [it]
        return tuple(res)

    def after_batch(self):
        "Take the stored results and puts it in `self.stats`"
        if self.training and (self.every is None or self.train_iter%self.every == 0): self.stats.append(self.hooks.stored)
        super().after_batch()

    def layer_stats(self, idx):
        lstats = self.stats.itemgot(idx)
        return L(lstats.itemgot(o) for o in ('mean','std','near_zero'))

    def hist(self, idx):
        res = self.stats.itemgot(idx).itemgot('hist')
        return torch.stack(tuple(res)).t().float().log1p()

    def color_dim(self, idx, figsize=(10,5), ax=None):
        "The 'colorful dimension' plot"
        res = self.hist(idx)
        if ax is None: ax = subplots(figsize=figsize)[1][0]
        ax.imshow(res, origin='lower')
        ax.axis('off')

    def plot_layer_stats(self, idx):
        _,axs = subplots(1, 3, figsize=(12,3))
        for o,ax,title in zip(self.layer_stats(idx),axs,('mean','std','% near zero')):
            ax.plot(o)
            ax.set_title(title)

In [ ]:
learn = synth_learner(n_trn=5, cbs = ActivationStats(every=4))
learn.fit(1)

[0, 7.943600177764893, 8.535039901733398, '00:00']


In [ ]:
learn.activation_stats.stats

(#2) [[{'mean': 1.3028467893600464, 'std': 0.32002925872802734, 'near_zero': 0.0}],[{'mean': 1.3026641607284546, 'std': 0.29966112971305847, 'near_zero': 0.0}]]

The first line contains the means of the outputs of the model for each batch in the training set, the second line their standard deviations.

In [ ]:
#| hide
def test_activation_stats_include_paramless(include_paramless=False):
    "create a learner, fit, then check number of layers"
    modl = nn.Sequential(nn.Linear(1,50), nn.ReLU(), nn.BatchNorm1d(50), nn.Linear(50, 1), nn.Flatten())

    learn = synth_learner(model=modl, cbs=ActivationStats(every=4, include_paramless=include_paramless))
    learn.fit(1)
    
    expected_stats_len = 3  
    if include_paramless: expected_stats_len = 5 # includes Relu & Flatten
    test_eq(expected_stats_len, len(learn.activation_stats.modules))

test_activation_stats_include_paramless(include_paramless=True)
test_activation_stats_include_paramless(include_paramless=False)

[0, 10.29939079284668, 6.808271408081055, '00:00']
[0, 12.69525146484375, 8.763984680175781, '00:00']


In [ ]:
def test_every(n_tr, every):
    "create a learner, fit, then check number of stats collected"
    learn = synth_learner(n_trn=n_tr, cbs=ActivationStats(every=every))
    learn.fit(1)
    expected_stats_len = math.ceil(n_tr / every)
    test_eq(expected_stats_len, len(learn.activation_stats.stats))
    
for n_tr in [11, 12, 13]:
    test_every(n_tr, 4)
    test_every(n_tr, 1)

[0, 7.132676601409912, 6.505333423614502, '00:00']
[0, 30.60495376586914, 29.395254135131836, '00:00']
[0, 14.507355690002441, 10.65038013458252, '00:00']
[0, 12.470440864562988, 7.216660499572754, '00:00']
[0, 30.247482299804688, 25.165172576904297, '00:00']
[0, 6.672229290008545, 5.598482131958008, '00:00']


In [ ]:
#| hide
class TstCallback(HookCallback):
    def hook(self, m, i, o): return o
    def before_fit(self):
        super().before_fit()
        self.means,self.stds = [],[]
    
    def after_batch(self):
        if self.training:
            self.means.append(self.hooks.stored[0].mean().item())
            self.stds.append (self.hooks.stored[0].std() .item())

learn = synth_learner(n_trn=5, cbs = [TstCallback(), ActivationStats()])
learn.fit(1)
test_eq(learn.activation_stats.stats.itemgot(0).itemgot("mean"), learn.tst.means)
test_eq(learn.activation_stats.stats.itemgot(0).itemgot("std"),  learn.tst.stds)

[0, 8.516677856445312, 11.47872543334961, '00:00']


## Export -

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()